
# **1. Pooling in CNNs**

Pooling layers reduce the **spatial resolution** of feature maps (height and width), allowing networks to:

* Increase **receptive field**
* Reduce **computation and memory cost**
* Introduce **translation invariance**
* Prevent **overfitting** by summarizing local features

---

## **1.1. Types of Pooling Layers in CNNs**

| Type                             | Formula                                                             | Intuition                                            | When to Use                                                                | Impact                                                                 |                                                            |                                            |
| -------------------------------- | ------------------------------------------------------------------- | ---------------------------------------------------- | -------------------------------------------------------------------------- | ---------------------------------------------------------------------- | ---------------------------------------------------------- | ------------------------------------------ |
| **Max Pooling**                  | $$y = \max(x_{i,j})$$                                               | Keeps the strongest activation in each region        | Most common (default) choice in CNNs                                       | Keeps salient edges/features; robust to noise; may discard subtle info |                                                            |                                            |
| **Average Pooling**              | $$y = \frac{1}{N}\sum_{i,j}x_{i,j}$$                                | Averages features in each region                     | When smooth feature maps are desired (e.g., before classification)         | Reduces noise, but blurs sharp features                                |                                                            |                                            |
| **Global Average Pooling (GAP)** | $$y_c = \frac{1}{HW}\sum_{i,j}x_{i,j,c}$$                           | Average over the whole feature map per channel       | Replaces fully connected layer at the end (e.g., ResNet)                   | Forces each channel to represent a class concept; fewer parameters     |                                                            |                                            |
| **Global Max Pooling (GMP)**     | $$y_c = \max_{i,j}(x_{i,j,c})$$                                     | Take the max across the entire spatial map           | Used when only the most prominent feature matters (e.g., object detection) | High selectivity; ignores global context                               |                                                            |                                            |
| **Lp Pooling**                   | $$y = \left(\frac{1}{N}\sum                                         | x                                                    | ^p\right)^{1/p}$$                                                          | Generalization of average/max pooling                                  | Can control between average ($p=1$) and max ($p\to\infty$) | Adjustable sensitivity to high activations |
| **Stochastic Pooling**           | Select random value from region proportional to activation strength | Adds stochasticity for regularization                | Used occasionally for regularization (rare today)                          | Acts as a regularizer; less deterministic                              |                                                            |                                            |
| **Adaptive Pooling**             | Output a target size regardless of input shape                      | Used in PyTorch `AdaptiveAvgPool2d` before FC layers | Ensures consistent output shape                                            | Flexible and shape-agnostic                                            |                                                            |                                            |

---

## **1.2. When to Use Which (CNN)**

| Scenario                                     | Recommended Pooling                    | Reason                                          |
| -------------------------------------------- | -------------------------------------- | ----------------------------------------------- |
| Image classification (ResNet, VGG)           | Max pooling early, then GAP at the end | Keeps strong features, ends with global context |
| Smooth/semantic tasks (segmentation)         | Average pooling                        | Avoid losing weak signals                       |
| Detection or localization                    | Max pooling                            | Preserves strong activations                    |
| Architecture flexibility (input size varies) | Adaptive Avg Pool                      | Consistent final feature vector                 |
| Regularization or stochastic effects         | Stochastic Pooling                     | Adds randomness; rarely used today              |

---

# **2. Pooling in Vision Transformers (ViTs)**

Transformers **don’t use pooling layers** in the classical sense — but they still **downsample** or **aggregate tokens** to reduce sequence length or extract global representation.

---

## **2.1. Pooling Analogues in ViTs**

| Pooling Equivalent                       | Where It Happens                  | Formula/Mechanism                                  | Impact                                                  | When to Use                                                             |                                                   |
| ---------------------------------------- | --------------------------------- | -------------------------------------------------- | ------------------------------------------------------- | ----------------------------------------------------------------------- | ------------------------------------------------- |
| **CLS Token pooling**                    | End of ViT (BERT-style)           | Output of `[CLS]` token represents the whole image | Learns a global representation during training          | Default ViT (classification tasks)                                      |                                                   |
| **Global Average Pooling over tokens**   | Alternative to CLS                | $$y = \frac{1}{N}\sum_i T_i$$                      | Averages all patch tokens                               | More stable; used in ConvNeXt-ViT hybrids                               |                                                   |
| **Patch Merging (Hierarchical Pooling)** | Swin Transformer                  | Concatenates and linearly projects nearby patches  | $$X' = W [X_{i,j}, X_{i+1,j}, X_{i,j+1}, X_{i+1,j+1}]$$ | Reduces resolution and increases feature dim; like CNN stride-2 pooling | When building hierarchical (pyramid) Transformers |
| **Token Pooling / Attention Pooling**    | DeiT, PoolFormer, Class-Attention | Attention weights aggregate tokens adaptively      | $$y = \text{softmax}(A)X$$                              | Adaptive pooling via attention weights                                  | Used in hybrid attention architectures            |
| **Spatial Reduction Attention (SRA)**    | PVT (Pyramid Vision Transformer)  | Reduces K,V token count by strided pooling         | Reduces attention cost (quadratic → linear)             | Used for efficiency on large images                                     |                                                   |
| **Adaptive Token Sampling / Pooling**    | TokenLearner, MobileViT           | Selects most informative tokens                    | Dynamic spatial pooling                                 | Used for efficient ViTs on edge devices                                 |                                                   |

---

## **2.2. When to Use Which (ViT)**

| Scenario                        | Recommended Pooling / Downsampling       | Reason                                                     |
| ------------------------------- | ---------------------------------------- | ---------------------------------------------------------- |
| **Standard ViT (224×224)**      | CLS Token                                | Simple and effective for classification                    |
| **More stable representation**  | Global Average Pool over tokens          | Reduces dependency on CLS token learning                   |
| **Hierarchical representation** | Patch Merging (Swin, PVT)                | Enables pyramid features (good for detection/segmentation) |
| **Lightweight / Mobile ViTs**   | Token Pooling (TokenLearner, PoolFormer) | Efficient adaptive token reduction                         |
| **Large-resolution inputs**     | Spatial Reduction Attention              | Reduces memory and compute cost                            |

---

# **3. Key Insights (CNN vs ViT)**

| Aspect                       | CNN                             | ViT                              |
| ---------------------------- | ------------------------------- | -------------------------------- |
| Pooling type                 | Fixed (max/avg)                 | Learnable / attention-based      |
| Purpose                      | Reduce spatial size             | Reduce token sequence length     |
| Global representation        | Global Average Pool             | CLS or mean token pooling        |
| Hierarchical feature pyramid | Natural (due to stride/pooling) | Explicitly added (Patch Merging) |
| Inductive bias               | Translation invariance          | None (learned)                   |

---

# **4. Summary of Best Practices**

✅ **CNNs**

* Use **MaxPool** in early layers for invariance.
* Use **GlobalAvgPool** instead of FC layers for compactness.
* Use **AdaptiveAvgPool** to handle variable input sizes.

✅ **ViTs**

* Use **CLS token** for classification tasks.
* Use **Patch Merging** (Swin-like) for detection/segmentation.
* Use **Global Average pooling** over tokens for more stable performance.
* Use **Attention-based pooling** for adaptive or efficient ViT variants.

---

Would you like me to show a **PyTorch code comparison** between

1. CNN pooling (MaxPool2d, AvgPool2d, GlobalAvgPool2d)
   and
2. ViT token pooling (CLS, mean pooling, patch merging)?
   It would help visualize the differences concretely.


## **small numeric example**
Let’s make a **small numeric example** that directly contrasts **pooling in CNNs** vs **token pooling in ViTs**, so you can *see* how each works.

We’ll use very small numbers for clarity.

---

# **1. CNN Pooling Example**

Assume we have a **feature map** (1 channel, 4×4):

$$
X =
\begin{bmatrix}
1 & 3 & 2 & 4 \
5 & 6 & 7 & 8 \
3 & 2 & 9 & 1 \
0 & 4 & 5 & 2
\end{bmatrix}
$$

---

### **1.1. Max Pooling (2×2, stride=2)**

We divide the image into non-overlapping 2×2 windows:

| Region       | Values    | Max |
| ------------ | --------- | --- |
| Top-left     | [1,3,5,6] | 6   |
| Top-right    | [2,4,7,8] | 8   |
| Bottom-left  | [3,2,0,4] | 4   |
| Bottom-right | [9,1,5,2] | 9   |

Resulting pooled feature map:

$$
Y_{\text{max}} =
\begin{bmatrix}
6 & 8 \
4 & 9
\end{bmatrix}
$$

✅ **Impact**: Keeps only strongest features (edges, corners).
❌ Loses weak but possibly useful information.

---

### **1.2. Average Pooling (2×2, stride=2)**

Average each 2×2 region instead of taking max:

| Region       | Values    | Mean |
| ------------ | --------- | ---- |
| Top-left     | [1,3,5,6] | 3.75 |
| Top-right    | [2,4,7,8] | 5.25 |
| Bottom-left  | [3,2,0,4] | 2.25 |
| Bottom-right | [9,1,5,2] | 4.25 |

Result:

$$
Y_{\text{avg}} =
\begin{bmatrix}
3.75 & 5.25 \
2.25 & 4.25
\end{bmatrix}
$$

✅ **Impact**: Smooths the representation.
❌ Reduces contrast between regions.

---

### **1.3. Global Average Pooling (entire 4×4)**

$$
y = \frac{1}{16}\sum X = \frac{68}{16} = 4.25
$$

✅ **Impact**: Converts full spatial map into a single scalar (per channel).
Used before classification heads.

---

# **2. ViT Pooling Example**

Now let’s simulate Vision Transformer pooling with **4 patch tokens + CLS token**.

Assume:
$$
\text{CLS} = [1, 1, 1], \quad
T_1 = [2, 0, 1], \quad
T_2 = [0, 3, 1], \quad
T_3 = [1, 2, 2], \quad
T_4 = [3, 1, 0]
$$

---

### **2.1. CLS Token Pooling (standard ViT)**

The CLS token attends to all patches during training, and we take only its embedding after the final transformer block.

So output = `[CLS]` = `[1, 1, 1]` → later becomes `[y₁, y₂, y₃]` (after MLP head).

✅ **Impact**: Learns a *weighted summary* of all tokens.

---

### **2.2. Mean Pooling over Tokens**

We can compute the mean over **all patch tokens (not CLS)**:

$$
\bar{T} = \frac{1}{4}(T_1 + T_2 + T_3 + T_4)
$$

Compute elementwise:

$$
T_1 + T_2 + T_3 + T_4 =
[2+0+1+3, ; 0+3+2+1, ; 1+1+2+0] = [6, 6, 4]
$$

So

$$
\bar{T} = [1.5, 1.5, 1.0]
$$

✅ **Impact**: Similar to Global Average Pooling in CNNs.
More stable than CLS-based pooling for small datasets.

---

### **2.3. Patch Merging (Swin Transformer–style)**

Imagine tokens come from 2×2 grid → [T₁, T₂, T₃, T₄].

Patch merging concatenates and projects them:

$$
\text{Concat}(T_1, T_2, T_3, T_4) =
[2,0,1, ; 0,3,1, ; 1,2,2, ; 3,1,0]
$$

Then we apply a linear layer (say, sum all values for simplicity):

$$
Y = 2+0+1+0+3+1+1+2+2+3+1+0 = 16
$$

Result: one merged token that represents the 2×2 region.

✅ **Impact**: Reduces token count, increases feature dimension (hierarchical).
❌ May lose fine-grained detail.

---

# **3. Summary Table**

| Operation           | Input Size | Output Size | Formula               | Effect                  |
| ------------------- | ---------- | ----------- | --------------------- | ----------------------- |
| Max Pool (CNN)      | 4×4        | 2×2         | $\max$ per region     | Keeps strongest signal  |
| Avg Pool (CNN)      | 4×4        | 2×2         | Mean per region       | Smooths representation  |
| Global Avg (CNN)    | 4×4        | Scalar      | $\frac{1}{HW}\sum x$  | Compresses spatial info |
| CLS Pool (ViT)      | N tokens   | 1 token     | Learned summary       | Learns global rep       |
| Mean Pool (ViT)     | N tokens   | 1 token     | $\frac{1}{N}\sum T_i$ | Simple global summary   |
| Patch Merging (ViT) | 4 tokens   | 1 token     | Concat + Linear       | Downsamples hierarchy   |

---




#  What is Max Pooling?
Max pooling is a key concept in **deep learning**, especially in **Convolutional Neural Networks (CNNs)** used for image processing and computer vision. 


**Max pooling** is a **downsampling** operation that reduces the spatial dimensions (width and height) of an input feature map while retaining the most important information.

It works by sliding a window (typically 2×2) over the input and **taking the maximum value** in each region.

---

### Example

Suppose you have a 4×4 feature map:

```
1  3  2  4  
5  6  1  2  
3  2  0  1  
1  2  4  3  
```

Applying 2×2 max pooling with stride 2 gives:

```
6  4  
3  4  
```

We took the **maximum** from each 2×2 block:
- max(1, 3, 5, 6) = 6
- max(2, 4, 1, 2) = 4
- etc.

---

###  Why Do We Use Max Pooling?

1. **Dimensionality Reduction**
   - Reduces the number of computations in later layers.
   - Helps with overfitting by summarizing regions.

2. **Translation Invariance**
   - Small shifts or movements in the image don’t change the pooled value.
   - Useful for recognizing features regardless of their exact position.

3. **Highlighting Strong Features**
   - Max pooling keeps only the **strongest activation** (most important signal) in each region.

---

###  Common Pooling Types

| Type           | What it does                         |
|----------------|--------------------------------------|
| Max Pooling    | Takes the **maximum** value          |
| Average Pooling| Takes the **average** value          |
| Global Pooling | Takes max/average over entire map    |

---

###  Typical Parameters

- **Kernel Size**: Size of the window (e.g., 2×2)
- **Stride**: How far the window moves (e.g., 2 skips every other pixel)
- **Padding**: Whether to pad the input to keep the same size (usually not used in pooling)

---

###  Is Max Pooling Always Good?

- **Pros**:
  - Reduces memory and computation
  - Adds robustness to small changes
  - Helps generalization

- **Cons**:
  - Can lose spatial precision
  - Not learnable (fixed operation)

---

###  Alternatives to Max Pooling

- **Strided Convolutions**: Learnable and can replace pooling
- **Global Average Pooling**: Often used before fully connected layers
- **Attention Mechanisms**: Learn what to focus on instead of blindly pooling

---

###  Intuition

Imagine scanning a patch of an image: max pooling keeps **only the strongest signal** (like the brightest pixel or most confident feature), making the model focus on **what matters most** while ignoring noise.

---

# The Order of Relu and Max Pooling 


The order **does matter**, and typically we use **ReLU → MaxPooling (most common)**, not the other way around.


With **ReLU** and **MaxPool**, the **forward result cannot differ**—for any pooling window $S$,

$$
\max(\operatorname{ReLU}(S)) \;=\; \operatorname{ReLU}(\max(S)).
$$

ReLU is monotone, and max is monotone, so the two orders give the same pooled value.

But you *can* get **different arg-max indices** (and thus different backprop routes / unpooling behavior) even though the numeric output matches. Here’s a concrete numeric case:

### Example (2×2 window)

Let the conv output in a pooling window be

$$
W=\begin{bmatrix}
-4 & -4\\
-4 & \mathbf{-1}
\end{bmatrix}
$$

* **Order A: Conv → ReLU → MaxPool**
  After ReLU:

  $$
  \operatorname{ReLU}(W)=\begin{bmatrix}
  0 & 0\\
  0 & 0
  \end{bmatrix}
  $$

  All entries tie at 0. Many libraries break ties by picking the **first** index (e.g., top-left).
  **Pooled value:** $0$. **Index chosen:** (top-left).

* **Order B: Conv → MaxPool → ReLU**
  MaxPool (pre-ReLU) picks the **least negative** (i.e., the maximum) which is $-1$ at **bottom-right**.
  After ReLU: $\operatorname{ReLU}(-1)=0$.
  **Pooled value:** $0$. **Index chosen:** (bottom-right).

So:

* **Forward pooled value** is $0$ in both orders (identical).
* **Selected index differs** (top-left vs bottom-right).
  This can change **which spatial location receives gradient** (or the stored indices used by MaxUnpool), even though the scalar output is the same. (In this specific all-nonpositive case, the gradient magnitude still becomes 0 due to ReLU, but the *index* recorded by pooling differs.)


---

#### Learnable parameters

* **Conv layers** have learnable parameters:

  * **weights** (the filter kernels)
  * **biases** (if enabled)
* **ReLU** has no parameters (it’s just a fixed non-linearity).
* **MaxPool** has no parameters either — it’s just an operation that selects the maximum in each window.

 So the *only* learnable parameters are in the convolution. Pooling never has learnable parameters.

---

#### What MaxPool does “remember”

MaxPool does not learn, but it **remembers the index of the max element** in each pooling window during the forward pass.

* In the **backward pass**, the gradient is sent only to that max location, all other positions get zero gradient.
* This is what we saw in the PyTorch experiment: the gradient routes depend on which index was picked.

---

#### Why order matters

Even though forward outputs from `Conv→ReLU→MaxPool` and `Conv→MaxPool→ReLU` are the same, the **chosen indices can differ** (especially in windows with negatives).

That means:

* **Gradient routing can differ**:

  * `Conv→ReLU→MaxPool`: negatives are zeroed first, so only positive activations can receive gradient.
  * `Conv→MaxPool→ReLU`: MaxPool might pick a negative as the “max” (if all are negative). After ReLU, the output becomes 0, but the index is still recorded → during backprop, the gradient will flow to that negative conv output before being squashed.
* This affects how the optimizer updates the conv **weights**, since gradients are computed with respect to those weights.

So, **the difference is not because MaxPool has learnable parameters** (it doesn’t).
The difference is in **which conv weights get updated**, because gradient flow is determined by the pooling index selection.

---

**Conclusion:**

* The learnable parameters are **conv weights & biases only**.
* Order matters because **gradient paths differ** due to how MaxPool selects indices **before or after ReLU**.
* Over many updates, this can slightly change how the network learns — which is why almost all architectures standardize on **Conv → ReLU → MaxPool**.

---



a **step-by-step backprop** with tiny 2×2 windows to show exactly what happens. We’ll use a single conv feature map (so we can ignore multi-channel complications), a **2×2 MaxPool** (so it reduces to a single scalar), and the loss $L$ is just the pooled output (so $\partial L/\partial(\text{pooled})=1$).

---

#### 1) All-negative window → gradients die in both orders

Let the **conv output** (pre-ReLU) be

$$
Z=\begin{bmatrix}
-4 & -2\\
-3 & -1
\end{bmatrix}
$$

**Order A: Conv → ReLU → MaxPool**

1. **ReLU**: $A=\operatorname{ReLU}(Z)=\begin{bmatrix}0&0\\0&0\end{bmatrix}$
2. **MaxPool** over $A$: pooled value $y = \max(A)=0$.
   (There’s a tie; suppose the pool **stores** top-left index, $(0,0)$, by convention.)
3. **Loss**: $L = y$ ⇒ $\frac{\partial L}{\partial y}=1$.

**Backward:**

* Through MaxPool: gradient goes to the stored index in $A$:
  $\frac{\partial L}{\partial A}=\begin{bmatrix}1&0\\0&0\end{bmatrix}$.
* Through ReLU: $A=\operatorname{ReLU}(Z)\Rightarrow \operatorname{ReLU}'(Z)=0$ element-wise (since all $Z\le 0$).
  $\frac{\partial L}{\partial Z}=\frac{\partial L}{\partial A}\odot \operatorname{ReLU}'(Z)=\mathbf{0}$.

**Result:** $\frac{\partial L}{\partial Z}=0$ everywhere ⇒ **no weight updates** from this window.

#### Order B: Conv → MaxPool → ReLU

1. **MaxPool** over $Z$: pooled pre-ReLU value $y'=\max(Z)=-1$ at index $(1,1)$ (bottom-right).
2. **ReLU**: $y=\operatorname{ReLU}(y')=\operatorname{ReLU}(-1)=0$.
3. **Loss**: $L=y$ ⇒ $\frac{\partial L}{\partial y}=1$.

**Backward:**

* Through ReLU at $y'=-1$: $\operatorname{ReLU}'(y')=0$ ⇒ $\frac{\partial L}{\partial y'}=0$.
* Through MaxPool: the gradient to $Z$ at the max index is $0$, others $0$ too.
  $\frac{\partial L}{\partial Z}=\mathbf{0}$.

**Result:** again **no weight updates**.

**Conclusion (all-negative case):** Picking $-1$ (Order B) vs a top-left tie at 0 (Order A) **does not help** — ReLU’s derivative is 0 at negatives, so gradients die either way.

---

#### 2) Mixed-sign window → gradients flow (and are the same)

Now let

$$
Z=\begin{bmatrix}
-4 & \mathbf{2}\\
1 & -3
\end{bmatrix}
$$

* **Order A (ReLU first):** $A=\begin{bmatrix}0&2\\1&0\end{bmatrix}$, MaxPool picks $2$ at $(0,1)$.
* **Order B (Pool first):** Max of $Z$ is $2$ at $(0,1)$; ReLU keeps it $2$.

Both orders: $y=2$, $L=y\Rightarrow \partial L/\partial y=1$.

**Backward:**

* The pool **stores the same index** $(0,1)$.
* ReLU derivative at that location is **1** (since $Z_{0,1}=2>0$).
* So $\frac{\partial L}{\partial Z}$ is **1 at $(0,1)$** and **0 elsewhere** — **for both orders**.

**Result:** **Same forward, same gradient flow** when there’s a strictly positive max.

---

#### Takeaways

* **MaxPool has no learnable parameters.** It only stores **indices** of maxima.
* With **all negatives**, both orders yield **zero gradient** due to $\operatorname{ReLU}'=0$ at negatives.
* With **positives present**, both orders pick the same (strict) max and route **the same gradient**.
* Differences can occur in **stored indices** for the all-nonpositive case (tie vs least negative), but **gradients still end up zero**.
* That’s why the community standardizes on **Conv → ReLU → MaxPool**: cleaner semantics (pool only over active features) without risking odd index choices that don’t help learning.
